# Natural image decomposition and hierarchical clustering
### Lauren E. Wool, Summer Workshop on the Dynamic Brain (September 2, 2017)

#### This code imports the Brain Observatory stimulus set of natural images, and decomposes them based on a preset bank of Gabor filters. Once decomposed, the convolved images are clustered using HCA or K-means clustering into "similar" groups. These groupings inform a new index by which to order the image indices for the representational similarity matrices (RSMs). 
#### As an alternative, the convolved images are clustered based on 'relevant pixels'—that is, the areas of the images that are 'seen' by the population receptive field. This produces a new image index for each experiment, by which the RSMs can be shuffled. 
#### If the new indexing order is somehow more informative that the arbitrary starting order, one would expect to see more 'clumping' in the RSM (similar to how neuron correlation 'clumps' in the static-grating RSMs over similar SFs, orientations, etc.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Imports, drive path, functions</h2>

<p>Set up your environment with the modules and functions you need
</p>
</div>

In [1]:
# Imports
import numpy as np
import pandas as pd
import os
import sys
import h5py
import cv2
from skimage import filters
from scipy.ndimage.filters import convolve
from scipy.misc import imresize
import swdb2017.brain_observatory.utilities.Plot_cell_rf_and_image as crfimage
import swdb2017.brain_observatory.utilities.Plot_population_rf_and_image as prfimage
import swdb2017.brain_observatory.get_brain_observatory_expdata as gboe
import allensdk.brain_observatory.stimulus_info as si
from allensdk.brain_observatory.observatory_plots import plot_mask_outline
from scipy import signal
from scipy import misc
import timeit
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from scipy import stats

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib notebook

# AWS
drive_path = '/data/dynamic-brain-workshop/brain_observatory_cache/'
# drive_path = '/Volumes/Brain2017/data/dynamic-brain-workshop/brain_observatory_cache/'

# Import the Brain Observatory cache from allensdk.core.brain_observatory_cache
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

manifest_file = os.path.join(drive_path,'brain_observatory_manifest.json')
print manifest_file

boc = BrainObservatoryCache(manifest_file=manifest_file)

/data/dynamic-brain-workshop/brain_observatory_cache/brain_observatory_manifest.json


In [2]:
# Functions

def fft_convolve(image,g_filter):
    conv = signal.fftconvolve(image, g_filter, mode='same')
    image_height = image.shape[0]
    image_length = image.shape[1]
    crop0 = int(image.shape[0]*.1)
    crop1 = int(image.shape[1]*.1)
    conv_cropped = conv[crop0:image_height-crop0,crop1:image_length-crop1]
    amp = (np.amax(np.abs(conv_cropped)))
    amp2 = np.amax(conv_cropped.real)-np.amin(conv_cropped.real)
    return conv_cropped, amp, amp2

def fft_convolve_no_crop(image,g_filter):
    conv = signal.fftconvolve(image, g_filter, mode='same')
    amp = (np.amax(np.abs(conv)))
    amp2 = np.amax(conv.real)-np.amin(conv.real)
    return conv, amp, amp2

def fft_convolve_amp_only(image,g_filter):
    conv = signal.fftconvolve(image, g_filter, mode='full')
    amp = (np.amax(np.abs(conv)))
    amp2 = np.amax(conv.real)-np.amin(conv.real)
    return amp, amp2

def downsample_image(image, percent):
    image_height = int(image.shape[0])
    image_length = int(image.shape[1])
    p = percent/100.
    im_ds = imresize(image, percent, interp='nearest', mode=None)
    return im_ds
    
def exp_id_tool(number):
    if number > 199:
        return exp_ids.index(number)
    else: 
        return exp_ids[number]

def scene_and_experiment_pop_rf(boc, experiment_id):
    pop_rf = get_population_rf(boc, experiment_id)
    m = si.BrainObservatoryMonitor()
    pop_rf[pop_rf>0]=1
    rf_convert = m.lsn_image_to_screen(pop_rf, origin='upper')
    rf_convert[np.where(rf_convert<200)]=0
    return rf_convert
    
def get_natural_scene_template_expt(boc, experiment_id):
    ns_session_id = boc.get_ophys_experiments(experiment_container_ids=[experiment_id], 
                                              stimuli=['natural_scenes'])[0]['id']
    data_set_ns = boc.get_ophys_experiment_data(ophys_experiment_id=ns_session_id)
    ns_template = data_set_ns.get_stimulus_template('natural_scenes')
    return ns_template

def get_population_rf(boc, experiment_id):
    c_flag = 'C'
    lsn_name = 'locally_sparse_noise'
    rf_name = 'receptive_field_lsn'

    for a in boc.get_ophys_experiments(experiment_container_ids=[experiment_id]):
        if a['session_type'].endswith('2'):
            c_flag = 'C2'
            if a['targeted_structure'] != 'VISp':
                lsn_name = 'locally_sparse_noise_8deg'
                rf_name = 'receptive_field_lsn8'
            else:
                lsn_name = 'locally_sparse_noise_4deg'
                rf_name = 'receptive_field_lsn4'

    drive_path = boc.manifest.get_path('BASEDIR')
    if c_flag=='C':
        session_id = boc.get_ophys_experiments(experiment_container_ids=[experiment_id], stimuli=[lsn_name])[0]['id']
        analysis_file = os.path.join(drive_path, 'ophys_experiment_analysis', str(session_id)+'_three_session_C_analysis.h5')
    elif c_flag=='C2':    
        session_id = boc.get_ophys_experiments(experiment_container_ids=[experiment_id], stimuli=[lsn_name])[0]['id']
        analysis_file = os.path.join(drive_path, 'ophys_experiment_analysis', str(session_id)+'_three_session_C2_analysis.h5')
    
    f = h5py.File(analysis_file, 'r')
    receptive_field = f['analysis'][rf_name].value
    f.close()
    pop_rf = np.nansum(receptive_field, axis=(2,3))
    return pop_rf

def get_pop_rf_amps(boc, experiment_id, conv):
    conv_test = conv.real
    rf_convert = scene_and_experiment_pop_rf(boc, experiment_id)
    rf_resize = misc.imresize(rf_convert, 25)
    rf_resize = rf_resize/255
    pad_x = int(np.floor((rf_resize.shape[1]-conv_test.shape[1])/2))
    pad_y = int(np.floor((rf_resize.shape[0]-conv_test.shape[0])/2))
    npad = ((pad_y, pad_y+1), (pad_x, pad_x+1))
    conv_pad = np.pad(conv_test, pad_width=npad, mode='constant', constant_values=0)
    conv_filtered = conv_pad*rf_resize
    amp = np.amax(conv_filtered)-np.amin(conv_filtered)
    return conv_filtered, amp

In [ ]:
# Load the representational similarity matrix information and save it
rs = gboe.get_all_representational_similarity_matrices(boc, 'three_session_B', 'natural_scenes', drive_path)

import json 
from swdb2017.general_tools.json_encoder import MyEncoder
filename ='/home/laurenw/repsimmat.json'
with open(filename, 'w') as data_file:
    data_file.write(json.dumps(rs,filename,indent = 4, ensure_ascii = True, cls = MyEncoder))

In [3]:
# Load the representational similarity matrix data (if you've saved it as per above)

import json 
from swdb2017.general_tools.json_encoder import MyEncoder
filename ='/home/laurenw/repsimmat.json'
with open(filename,'r') as data_file:
    rs = json.load(data_file)

In [4]:
# Generate a 3D array of all the representational similarity matrices 

rs_all = np.zeros((199,118,118))
for i in range(len(rs['representational_similarity'])):
    r = np.array(rs['representational_similarity'][i])    
    r = r[1:119,1:119] #remove the blank image frame from the experiment's RSM
    np.fill_diagonal(r,np.nan)
    r[~np.isnan(r)] = stats.mstats.zscore(r[~np.isnan(r)])
    rs_all[i,:,:] = r  

In [5]:
# Retrieve information for each natural-scenes experiment
session_type = 'three_session_B'
exps = boc.get_experiment_containers(file_name=None, ids=None, 
                                         targeted_structures=None, 
                                         imaging_depths=None, 
                                         cre_lines=None, 
                                         transgenic_lines=None, 
                                         include_failed=False)
first_session_id = boc.get_ophys_experiments(
                experiment_container_ids=[exps[0]['id']], 
                session_types = [session_type])[0]['id']

# Generate a list of experiment IDs (experiment container ID) and session IDs (natural-scenes experiment ID)
exp_ids = []
session_ids = []
for e in range(len(exps)):
    exp_ids.append(exps[e]['id'])
    session_id = boc.get_ophys_experiments(
                experiment_container_ids=[exps[e]['id']], 
                session_types = [session_type])[0]['id']
    session_ids.append(session_id)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Initial image processing</h2>

<p>Load the images and create filters
</p>
</div>

In [ ]:
# Load the experimental data for a single experiment (any will do)
data_set = boc.get_ophys_experiment_data(501498760)

# Read in a 3D array of images (shape: n_images,image_height,image_length)
scenes = data_set.get_stimulus_template('natural_scenes')

# Downsample the images
p = 0.25
scenes_ds = np.zeros((scenes.shape[0],int(scenes.shape[1]*p),int(scenes.shape[2]*p)))
for sidx,s in enumerate(scenes):
    im_ds = downsample_image(s,25)
    scenes_ds[sidx] = im_ds

In [ ]:
# Create gabor filters
'''for 0.1 cycles/deg, that’s 10 degrees needed for a full cycle
293 px/96 deg =  3.1 px/deg (96 degrees = screen width)
96 deg/293px = 0.33 deg/px
if you need 10 degrees to display a full cycle, then you need 31 pixels
0.1 cycles/deg = 0.1 cycles/3.1 px = 0.03 cycles/px'''

sf = np.array([0.02, 0.04, 0.08, 0.16, 0.32]) # cycles/deg
ori = np.array([0., 30., 60., 90., 120., 150.]) # deg

# Correct the values based on the inputs required for skimage.filters
sf_rad = (sf/180)*np.pi
ori_rad = (ori/180)*np.pi
sf_px = sf/3.1 

# Global filter parameters
n_stds = 3
bw = 10

# Initialize the list of gabors
gabor_list = []

# Loop through the sf and ori parameter arrays and build the filters
for sidx,s in enumerate(sf_px):
    sigma_x = 1/sf_px[sidx] * .5
    sigma_y = 2*sigma_x
    for oidx,o in enumerate(ori_rad):
        # Build a gabor filter
        gabor = filters.gabor_kernel(s, o, bw, sigma_x, sigma_y, n_stds, 0)
        # Append it to "gabor_list"
        gabor_list.append(gabor)

In [ ]:
# Show a gabor-filtered image and the original image

image = scenes_ds[50]
gfilter = gabor_list[25]
output, amp, amp2 = fft_convolve_no_crop(image,gfilter)
print(amp2)
fig = plt.figure()
ax1 = fig.add_subplot(121),plt.imshow(image, cmap = 'gray')
ax2 = fig.add_subplot(122),plt.imshow(output.real, cmap = 'gray')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Image analysis: images only</h2>

<p>Convolutions and clustering analysis
</p>
</div>

In [ ]:
#Iteratively convolve all the filters with all the images

n_images = scenes_ds.shape[0]
values_array = np.zeros((n_images,len(gabor_list)))
all_convs = []
for sidx,s in enumerate(scenes_ds):
    for gidx,g in enumerate(gabor_list):
        a = s
        b = g
        conv, amp, amp2 = fft_convolve(a,b)
        values_array[sidx,gidx] = amp2
        all_convs.append(conv)

In [ ]:
# Do hierarchical clustering analysis of the image x filter array
# Then plot the dendrogram
Z = linkage(values_array, 'ward')
plt.figure()
plt.title('Clustering scenes by Gabor filters')
plt.xlabel('natural scene index')
plt.ylabel('cluster distance')
R = dendrogram(Z,leaf_rotation=90., leaf_font_size=6.)
plt.show()

In [ ]:
# Try k-means clustering too, to see what you get (you have to manually define how many clusters you want)

k = KMeans(n_clusters=4, random_state=0).fit_predict(values_array)
k_sorted = np.argsort(k)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Image analysis—receptive fields</h2>

<p>Mask the convolution arrays by the population receptive field for a given experiment, then assess the magnitude from that.
</p>
</div>

In [ ]:
#Iteratively convolve all the filters with all the images

n_images = scenes_ds.shape[0]
values_array = np.zeros((n_images,len(gabor_list)))
all_convs = []
for sidx,s in enumerate(scenes_ds):
    for gidx,g in enumerate(gabor_list):
        a = s
        b = g
        conv, amp, amp2 = fft_convolve_no_crop(a,b)
        values_array[sidx,gidx] = amp2
        all_convs.append(conv)

In [112]:
# Load the convolution magnitudes for all the experiments (contstrained by population RF in each)
# (If you haven't already done this, generate the arrays with the cell below)
rf_values_array = np.load('rf_values_array.npy')
rf_values_array_sums = np.load('rf_values_array_sums.npy')

In [ ]:
n_exps = len(exp_ids)
n_images = scenes_ds.shape[0]
n_filters = len(gabor_list)
rf_values_array = np.zeros((n_exps, n_images,n_filters))
rf_values_array_sums = np.zeros((n_exps, n_images,n_filters))

for eidx,e in enumerate(exp_ids):
    try:
        rf_convert = scene_and_experiment_pop_rf(boc, e)
        rf_resize = misc.imresize(rf_convert, 25)
        rf_resize = rf_resize/255
    except:
        rf_resize = np.zeros((300,480))
    print(eidx)
    for s in range(n_images):
        for g in range(n_filters):
            a = all_convs[s*len(gabor_list)+g].real
            pad_x = int(np.floor((rf_resize.shape[1]-a.shape[1])/2))
            pad_y = int(np.floor((rf_resize.shape[0]-a.shape[0])/2))
            npad = ((pad_y, pad_y+1), (pad_x, pad_x+1))
            conv_pad = np.pad(a, pad_width=npad, mode='constant', constant_values=0)
            conv_filtered = conv_pad*rf_resize
            amp = np.amax(conv_filtered)-np.amin(conv_filtered)
            ampsum = np.sum(abs(conv_filtered))
            rf_values_array[eidx,s,g] = amp
            rf_values_array_sums[eidx,s,g] = ampsum

np.save('rf_values_array.npy',rf_values_array)
np.save('rf_values_array_sums.npy',rf_values_array_sums)

In [113]:
# Do hierarchical clustering analysis of the image x filter slice of the 199-experiment array
# Each element of Z is the linkage info for each experiment
# Each element of R is the RSM shuffling index for each experiment
Z_rf = []
R_rf = []
for v in rf_values_array_sums:
    Z = linkage(v, 'ward')
    Z_rf.append(Z)
    R = dendrogram(Z, p=30, truncate_mode=None, color_threshold=None, 
                   get_leaves=True, orientation='top', labels=None, count_sort=False, 
                   distance_sort=False, show_leaf_counts=False, no_plot=True)
    R_rf.append(R['leaves'])
    

In [ ]:
# Examine the dendrogram for a particular experiment

Z = linkage(rf_values_array[63], 'ward')
plt.figure()
plt.title('Clustering scenes by Gabor filters')
plt.xlabel('natural scene index')
plt.ylabel('cluster distance')
R = dendrogram(Z,leaf_rotation=90., leaf_font_size=6.)
plt.show()

In [ ]:
# Try k-means clustering too, to see what you get (you have to manually define how many clusters you want)
# Each element in K_sorted is the RSM shuffling index for each experiment
K_sorted = []
for v in rf_values_array_sums:
    k = KMeans(n_clusters=4, random_state=0).fit_predict(v)
    K_sorted.append(np.argsort(k))

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Representational similarity matrix analysis: images only</h2>

<p>Compare the experimental matrix to a shuffled matrix based on the clustering done above
</p>
</div>

In [147]:
# Reshuffle the RSM based on a clustering output index
def shuffle_RSM(rs_all,idx):
    rs2 = np.copy(rs_all)
    idx1 = idx
    rs_Z = np.zeros((199,118,118))
    for ridx,r in enumerate(rs2):
        test_rs = np.copy(r)
        test_rs = test_rs[idx1,:]
        test_rs = test_rs[:,idx1]
        rs_Z[ridx,:,:] = test_rs
    return rs_Z

rs_Z = shuffle_RSM(rs_all, R['leaves'])
rs_K = shuffle_RSM(rs_all, k_sorted)


In [ ]:
# Plot the comparison of the clustering output versus the original RSM

def plot_RSM_comparison(original_rsm, rs_Z, rs_K, rs_index):
    plt.figure()
    plt.subplot(131),plt.imshow(original_rsm[rs_index],cmap='viridis',vmin=-2, vmax=2)
    plt.subplot(132),plt.imshow(rs_Z[rs_index],cmap='viridis',vmin=-2, vmax=2)
    plt.subplot(133),plt.imshow(rs_K[rs_index],cmap='RdBu',vmin=-2, vmax=2)

plot_RSM_comparison(rs_original, rs_Z, rs_K, g)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Representational similarity matrix analysis: RF-specific</h2>

<p>Compare the experimental matrix to a shuffled matrix based on the clustering done above
</p>
</div>

In [116]:
# Reshuffle the RSM based on a clustering output index
def shuffle_RSM_unique_rfs(rs_all,idx):
    rs_Z = np.zeros((199,118,118))
    for ridx,r in enumerate(rs_all):
        idx1 = idx[ridx]
        test_rs = r
        test_rs = test_rs[idx1,:]
        test_rs = test_rs[:,idx1]
        rs_Z[ridx,:,:] = test_rs
    return rs_Z

rs_Z = shuffle_RSM_unique_rfs(rs_all, R_rf)
rs_K = shuffle_RSM_unique_rfs(rs_all, K_sorted)

In [ ]:
def plot_RSM_comparison(original_rsm, rs_Z, rs_K, rs_index):
    plt.figure()
    plt.subplot(131),plt.imshow(original_rsm[rs_index], vmin=-2, vmax=2)
    plt.subplot(132),plt.imshow(rs_Z[rs_index],vmin=-2, vmax=2)
    plt.subplot(133),plt.imshow(rs_K[rs_index], vmin=-2, vmax=1.5)
    plt.savefig('RF' + str(rs_index) + '.pdf')

for i in range(199):    
    plot_RSM_comparison(rs_all, rs_Z, rs_K, i)
